In [1]:
import requests
import pandas as pd
import json
import time
import random

In [ ]:
def fetch_sec_facts_by_sic(industry_mapping, sic_code, headers, user_start, user_end, limit=None):
    """
    Fetch SEC XBRL company facts for companies with a specific SIC code.

    Args:
        company_data (pd.DataFrame): DataFrame with company info including 'SIC' and 'cik_str' columns.
        sic_code (int): The SIC code to filter companies by.
        headers (dict): Headers to use for the SEC API request (must include User-Agent).
        user_start (int): Starting fiscal year to filter data.
        user_end (int): Ending fiscal year to filter data.
        limit (int, optional): Limit the number of CIKs to fetch. Default is None (fetch all).

    Returns:
        pd.DataFrame: Filtered DataFrame containing SEC facts.
    """
    filtered_cik_list = industry_mapping[industry_mapping['2022 NAICS Code'].astype(str).str[:2] == sic_code]['cik_str'].tolist()
    ciks = filtered_cik_list if limit is None else filtered_cik_list[:limit]

    all_data = []

    for cik in ciks:
        try:
            response = requests.get(
                f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
                headers=headers
            )
            response.raise_for_status()
            data_dict = response.json()
            cik_val = data_dict.get('cik')
            entityName = data_dict.get('entityName')

            for taxonomy, fact_group in data_dict.get('facts', {}).items():
                for fact_name, fact_data in fact_group.items():
                    label = fact_data.get('label')
                    description = fact_data.get('description')
                    units_dict = fact_data.get('units', {})
                    for unit, records in units_dict.items():
                        for record in records:
                            row = {
                                'cik': cik_val,
                                'entityName': entityName,
                                'taxonomy': taxonomy,
                                'fact_name': fact_name,
                                'label': label,
                                'description': description,
                                'unit': unit
                            }
                            row.update(record)
                            all_data.append(row)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for CIK {cik}: {e}")
        except (KeyError, json.JSONDecodeError) as e:
            print(f"Error processing data for CIK {cik}: {e}")

        time.sleep(random.uniform(1, 2))  # Polite delay

    df = pd.DataFrame(all_data)

    # Convert 'start' and 'end' columns to datetime
    for col in ['start', 'end']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Apply filtering conditions
    if not df.empty:
        df = df[
            (df['fy'] >= user_start) &
            (df['fy'] <= user_end) &
            ((df['end'] - df['start']).dt.days > 350) &
            (df['end'].dt.year == df['fy'])
        ]

    return df

In [ ]:
headers = {'User-Agent': "mbambal@purdue.edu"}

industry_mapping_file = '/Users/mayankbambal/Desktop/10K_API/data/mapping/industrymapping.csv'

industry_mapping = pd.read_csv(industry_mapping_file)
# Get user input

naics = input("Enter 2 Digit NAICS code: ")
user_start = int(input("Enter start period (YYYY): "))
user_end = int(input("Enter end period (YYYY): "))

#Download data

data_10k = fetch_sec_facts_by_sic(industry_mapping, naics, headers, user_start, user_end, limit=5)


# Save data to csv
data_10k.to_csv(f'{naics}_{user_start}_{user_end}data_10k.csv', index=False)

In [ ]:
import pandas as pd


df = data_10k

# ------------------------------
# Define which fact_names should be labeled "Revenue" in the StandardName column
# ------------------------------
valid_labels = [
    "Revenue",
    "Revenue, Net (Deprecated 2018-01-31)",
    "Revenues",
    "Sales Revenue",
    "RevenueFromContractWithCustomerExcludingAssessedTax"
]

# ------------------------------
# Create a new column "StandardName" that defaults to the original fact_name
# For the 'valid_labels', overwrite StandardName to "Revenue".
# ------------------------------
df["StandardName"] = df["fact_name"]  # keep original KPI names by default
df.loc[df["fact_name"].isin(valid_labels), "StandardName"] = "Revenue"

# ------------------------------
# Ensure fiscal year (fy) exists or fallback to extracting year from 'end' date
# ------------------------------
if 'fy' not in df.columns or df['fy'].isnull().all():
    df['fy'] = pd.to_datetime(df['end'], errors='coerce').dt.year

# ------------------------------
# Define a function to get the range of years for each group
# ------------------------------
def year_range(fy_series):
    years = sorted(fy_series.dropna().unique())
    if len(years) == 0:
        return "N/A"
    return f"{years[0]} - {years[-1]}"

# ------------------------------
# Group by Company, CIK, StandardName, ReportedName and summarize years used
# ------------------------------
kpi_summary = (
    df.groupby(["entityName", "cik", "StandardName", "label"])
      .agg(Years_Used=('fy', year_range))
      .reset_index()
)

# ------------------------------
# Rename columns for the output
# ------------------------------
kpi_summary.rename(columns={
    "entityName": "Company",
    "cik": "CIK",
    "label": "ReportedName",
    "Years_Used": "Years Used"
}, inplace=True)

# ------------------------------
# Export to CSV
# ------------------------------
output_file = "Revenue_KPI_Mapping.csv"
kpi_summary.to_csv(output_file, index=False)
print(f"All KPI mapping (with revenue renamed) created and saved to {output_file}")